In [ ]:
import os
import sys

script_dir = os.getcwd()
root_dir = f"{script_dir}/../"
sys.path.append(os.path.join(root_dir, "dpc"))

import numpy as np
import scipy.io
import imageio

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

import open3d
from open3d import JVisualizer

from models import model_pc
from util.config import setup_config
from util.system import setup_environment
from util.euler import quaternion2euler
from util.image import preprocess_input_image
from render.render_point_cloud import render_point_cloud

In [ ]:
# setup the model and load the data

exp_dir = "chair_unsupervised"

config = {
    "gpu": 0,
    "config": f"{exp_dir}/config.yaml",
    "checkpoint_dir": exp_dir,
    "batch_size": 1,
    "align_to_canonical": True,
}

cfg = setup_config(config)

setup_environment(cfg)

cfg.batch_size = 1
cfg.step_size = 1

g = tf.Graph()
with g.as_default():
    model = model_pc.ModelPointCloud(cfg)
    inputs = tf.placeholder(dtype=tf.float32, shape=[cfg.batch_size, cfg.image_size, cfg.image_size, 3])
    outputs = model.model_predict(inputs)
    points = outputs["points_1"]
    camera_pose = outputs["pose_student"]
    
    config = tf.ConfigProto(
        device_count={'GPU': 1}
    )

    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    variables_to_restore = tf.contrib.framework.get_variables_to_restore(exclude=["meta"])

restorer = tf.train.Saver(variables_to_restore)
checkpoint_file = tf.train.latest_checkpoint(exp_dir)
print("restoring checkpoint", checkpoint_file)
restorer.restore(sess, checkpoint_file)
print("Done.")

In [ ]:
# select an input image from the validation set (splits defined in data/splits)
# the dataset has 5 different viewpoints for the same model
model_name = "708cd72e15dfaab8a687729fda50af77"
view_idx = 3

image_file_name = f"{root_dir}/data/renders/03001627/{model_name}/render_{view_idx}.png"
input_image_np = imageio.imread(image_file_name)
input_image_np, _ = preprocess_input_image(input_image_np)

# show input image
plt.imshow(input_image_np)

In [ ]:
# predict point cloud
(points_np, camera_pose_np) = sess.run([points, camera_pose],
                                       feed_dict={inputs: np.expand_dims(input_image_np, axis=0)})

# Interactive visualisation with Open3D
pcd = open3d.PointCloud()
pcd.points = open3d.Vector3dVector(np.squeeze(points_np))

visualizer = open3d.JVisualizer()
visualizer.add_geometry(pcd)
visualizer.show()

In [ ]:
# render the point cloud with the predicted camera pose
# on the left is the input image and on the right is predicted shape and pose

camera_pose_np /= np.linalg.norm(camera_pose_np)
ypr = quaternion2euler(np.squeeze(camera_pose_np))
yaw = np.degrees(ypr[0])
pitch = np.degrees(ypr[1])
roll = np.degrees(ypr[2])

render_cfg = {
    "vis_azimuth": yaw,
    "vis_elevation": pitch,
    "vis_dist": 2.0,
    "render_image_size": 512,
    "render_cycles_samples": 100
}

image = render_point_cloud(points_np, render_cfg)
f, axarr = plt.subplots(1, 2, figsize=(12, 12))
f.set_figheight(512)
axarr[0].imshow(input_image_np)
axarr[0].axis('off')
axarr[1].imshow(image)
axarr[1].axis('off')